In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.sql.functions import col,dense_rank,when,max
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("app").master("local[3]").getOrCreate()

In [0]:
schema = StructType([
    StructField("id",IntegerType(),False),
    StructField("salary",IntegerType(),False)
])
data = [
( 1  , 100 )   ,
( 2  , 200 )   ,
( 3  , 300 )   ,
( 4  , 400 )   ,
( 5  , 300 )   ,
( 6  , 200 )   
]
emp = spark.createDataFrame(data,schema)
emp.show()

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   200|
|  3|   300|
|  4|   400|
|  5|   300|
|  6|   200|
+---+------+



In [0]:
# Write a solution to find the nth highest salary from the Employee table. If there is no nth highest salary, return null.

# 2nd Highest salary
window_spec = Window.orderBy(col("salary").desc())

nth_value = 2
print(f"{nth_value} Highest Salary")
emp.withColumn("rnk",dense_rank().over(window_spec)).filter(col("rnk")==nth_value).select(max(col("salary")).alias(f"{nth_value}_highest_salary")).distinct().show()

nth_value = 7
print(f"{nth_value} Highest Salary")
emp.withColumn("rnk",dense_rank().over(window_spec)).filter(col("rnk")==nth_value).select(max(col("salary")).alias(f"{nth_value}_highest_salary")).distinct().show()


2 Highest Salary
+----------------+
|2_highest_salary|
+----------------+
|             300|
+----------------+

7 Highest Salary
+----------------+
|7_highest_salary|
+----------------+
|            null|
+----------------+



In [0]:
emp.createOrReplaceTempView("emp")

spark.sql("with cte as (select distinct salary,dense_rank() over(order by salary desc) rnk from emp) select max(salary) as 2nd_highest_Salary from cte where rnk=2").show()

spark.sql("with cte as (select distinct salary,dense_rank() over(order by salary desc) rnk from emp) select  max(salary) as 7th_highest_Salary from cte where rnk=7").show()

+------------------+
|2nd_highest_Salary|
+------------------+
|               300|
+------------------+

+------------------+
|7th_highest_Salary|
+------------------+
|              null|
+------------------+



In [0]:
spark.stop()